In [ ]:
# load data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [2]:
import tensorflow as tf

In [3]:
# softmax model

x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder(tf.float32, [None, 10])  # ground truth

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
# Note that in the source code, they don't use this formulation, because it is numerically unstable.
# Instead, they apply tf.nn.softmax_cross_entropy_with_logits on the unnormalized logits.
# (e.g, we call softmax_cross_entropy_with_logits on tf.matmul(x, W)+b), because this more numerically stable function
# internally computes the softmax activation. 
# In your code, consider using tf.nn.softmax_cross_entropy_with_logits instead.
# cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

learning_rate = 0.5
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

# launch the model in an InteractiveSession
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init)

for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_:batch_ys})
    
# evaluating model
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9185


In [4]:
### Build a Multilayer Convolutional Network

# Weight Initialization
# 1. we need to create a lot of weights and biases
# 2. One should generally initialize weights with a small amount of noise for symmetry breaking, and prevent 0 gradients.
# 3. Since we're using ReLU neurons, it is also good practice to initialize them with a slightly positive initial bias to avoid "dead neurons".
# So we create two handy functions to prevent doing this repeatedly.

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1) # with a small amount of noise for symmetry breaking
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# Convolution and Pooling
# In this exmple, we're always going to choose the vanilla version. 
# Our convolutions uses a stride of one and are zero padded so that the output is the same size as the input.

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], 
                          strides=[1, 2, 2, 1], padding='SAME')


x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])  # ground truth

# reshape x to a 4d tensor, with the second and third dimensions 
# corresponding to image width and height, and the final dimension corresponding to the number of color channels
x_image = tf.reshape(x, [-1, 28, 28, 1]) 


# First Convolutional Layer
W_conv1 = weight_variable([5, 5, 1, 32]) # compute 32 features for each 5x5 patch. input channel is 1, ouput channel is 32
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1) # reduce the image size to 14x14


# Second Convolutional Layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2) # reduce the image size to 7x7

# Densely Connected Layer
# Now that the image size has been reduced to 7x7, we add a fully-connected layer with 1024 neurons 
# to allow processing on the entire image. 
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Droput
keep_prob = tf.placeholder(tf.float32) # the probability that a neuron's output is kept during dropout
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Readout Layer
# Finally, we add a layer, just like for the one layer softmax regression above
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


# Train and Evaluate the Model
# The different are that:
#    1. We will replace the steepest gradient descent optimizer with the more sophisticated ADAM optimizer.
#    2. We will include the additional parameter keep_prob in feed_dict to control the dropout rate.
#    3. We will add logging to every 100th iteration in the training process.

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
learning_rate = 1e-4
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# we will use tf.Session rather than tf.InteractiveSession. This better separates the process of creating the graph
# (model specification) and the process of evaluating the graph (model fitting).
# It generally makes for cleaner code. The tf.Session is created within a with block so that it is automatically 
# destroyed once the block is exited.
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={
                    x: batch[0], y_: batch[1], keep_prob: 1.0})
            print('step %d, training accuracy %g' % (i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    
    print('test accuracy %g' % accuracy.eval(feed_dict={
                x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))


step 0, training accuracy 0.04
step 100, training accuracy 0.86
step 200, training accuracy 0.94
step 300, training accuracy 0.88
step 400, training accuracy 0.96
step 500, training accuracy 0.9
step 600, training accuracy 0.98
step 700, training accuracy 0.96
step 800, training accuracy 0.9
step 900, training accuracy 1
step 1000, training accuracy 0.98
step 1100, training accuracy 0.92
step 1200, training accuracy 0.98
step 1300, training accuracy 0.98
step 1400, training accuracy 0.96
step 1500, training accuracy 0.98
step 1600, training accuracy 1
step 1700, training accuracy 0.94
step 1800, training accuracy 0.96
step 1900, training accuracy 0.94
step 2000, training accuracy 0.98
step 2100, training accuracy 0.96
step 2200, training accuracy 0.96
step 2300, training accuracy 1
step 2400, training accuracy 1
step 2500, training accuracy 1
step 2600, training accuracy 0.98
step 2700, training accuracy 1
step 2800, training accuracy 0.98
step 2900, training accuracy 0.98
step 3000, t

ResourceExhaustedError: OOM when allocating tensor with shape[10000,28,28,32]
	 [[Node: Conv2D = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](Reshape, Variable/read)]]
	 [[Node: Mean_1/_7 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_58_Mean_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'Conv2D', defined at:
  File "/opt/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/opt/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/opt/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/opt/anaconda2/lib/python2.7/site-packages/tra